Installing the packages required for the Apriori algorithm

In [1]:
pip install mlxtend  


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install apyori

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Importing the necessary libraries here
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import csv
import itertools
from pprint import pprint
import os


In [4]:
#Loading the dataset "political_social_media.csv" into a dataframe called "table" and reading the contents
table = pd.read_csv('political_social_media.csv', encoding="utf-8")
table
table.head() 


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,...,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,...,0,0,0,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),0,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,...,0,0,0,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),0,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,...,0,0,0,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),0,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,...,0,0,0,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),0,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,...,0,0,0,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),0,twitter,.@amazon delivery #drones show need to update ...


In [5]:
#Exploring the columns of the data 
table.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'audience', 'audience:confidence', 'bias',
       'bias:confidence', 'message', 'message:confidence', 'orig__golden',
       'audience_gold', 'bias_gold', 'bioid', 'embed', 'id', 'label',
       'message_gold', 'source', 'text'],
      dtype='object')

In [6]:
#Exploring the different types of bias 
table.bias.unique() 

array(['partisan', 'neutral'], dtype=object)

In [7]:
#Dropping the columns that are not required for our analysis
table.drop([ '_unit_id','_golden', '_unit_state', '_trusted_judgments', '_last_judgment_at','orig__golden',
            'audience_gold', 'bias_gold', 'bioid', 'embed', 'id', 'label',
             'message_gold', 'audience', 'audience:confidence', 'source' ], axis = 1, inplace = True) 
table

,bias,bias:confidence,message,message:confidence,text
0,partisan,1.0,policy,1.0,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,partisan,1.0,attack,1.0,VIDEO - #Obamacare: Full of Higher Costs and ...
2,neutral,1.0,support,1.0,Please join me today in remembering our fallen...
3,neutral,1.0,policy,1.0,RT @SenatorLeahy: 1st step toward Senate debat...
4,partisan,1.0,policy,1.0,.@amazon delivery #drones show need to update ...
...,...,...,...,...,...
4995,partisan,1.0,support,1.0,I applaud Governor Perry‰Ûªs recent decision t...
4996,partisan,1.0,policy,1.0,"Today, I voted in favor of H.R. 5016 - Financi..."
4997,neutral,1.0,policy,1.0,(Taken from posted WOKV interview) Congressm...
4998,neutral,1.0,media,1.0,Join me next week for a town hall in Ocala! I'...


In [8]:
#Appling NLTK to count the number of neutral and partisan tweets contained in the dataset
import nltk
nltk.download('punkt')
top_N = 4
#if not necessary all lower
a = table['bias'].str.lower().str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(a)
word_dist = nltk.FreqDist(words)
print (word_dist)

rslt = pd.DataFrame(word_dist.most_common(top_N),
                    columns=['Word', 'Frequency'])
print(rslt)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\suchi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<FreqDist with 2 samples and 5000 outcomes>
       Word  Frequency
0   neutral       3689
1  partisan       1311


In [9]:
#Sorting the tweets by selecting the bias as partisan and grouping them by messages and bias:confidence
partisan = (table[table['bias'] =="partisan"] 
          .groupby(['text', 'message'])['bias:confidence'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('text')) 

In [10]:
#Displaying the sorted tweets
len(partisan)
partisan

message,attack,constituency,information,media,mobilization,other,personal,policy,support
text,,,,,,,,,
"""At issue is something that strikes at the very heart of who we are, what we believe &amp; what we have always stood for"" http://t.co/Abyv1a7p1e",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"""I will sign a universal health care bill into law by the end of my first term as president that will cover every American and cut the cost of a typical family's premium by up to $2,500 a year."" - Barack Obama, June 23, 2007",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"""IRS taxing of tanning beds and other #Obamacare absurdities"" - @WashingtonPost column: http://t.co/gBxx0uLlXF #TrainWreck",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"""Let me be clear, federal workers did not cause this shutdown and should not lose pay because of it."" -@brianschatz",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"""Obamacare will push the equivalent of about 2 million workers out of the labor market by 2017 as employees decide either to work fewer hours or drop out altogether, according to the latest estimates Tuesday from the Congressional Budget Office.""",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
"‰ÛÏObamacare wasn't competently run,‰Û� via @TheAtlantic. In @HouseCommerce hearings, we will look for #ObamcareAnswers. http://t.co/IcfqSryEZB",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
‰ÛÏPpl of faith shouldn‰Ûªt have their govt tell them how to practice their religion‰Û_#Obamacare was an historic mistake‰Û� http://t.co/AoXqpvkWrk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"‰ÛÏThe House has acted time and time again to avert a government shutdown and spare Americans from the pain Obamacare will undoubtedly cause. And tonight, we acted yet again and passed a sensible proposal that keeps the government open, gives individuals and families the same delay that employers have been afforded, and eliminates members‰Ûª employer contribution. Unfortunately, the Senate, so hellbent on protecting President Obama and his signature law, has rejected compromise outright. I am disappointed that the Senate has refused to listen to the American people and work with the House, but I am hopeful we will come together to keep the government‰Ûªs lights on.‰Û� http://womack.house.gov/news/documentsingle.aspx?DocumentID=351894",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
#We will only use the columns 'bias', 'message','text' to build the association rule
columns = ['bias', 'message','text']

In [12]:
not_used_columns = list(set(table.columns.to_list()) - set(columns))
table = pd.get_dummies(table, columns = columns)
table.drop(labels=not_used_columns, axis=1, inplace=True)

In [13]:
#Assigning the Apriori min support to 0.05
min_support = 0.05
#Max lenght of apriori n-grams
max_len = 2
frequent_items = apriori(table, use_colnames=True, min_support=min_support, max_len= max_len)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)
rules.head(100).sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(message_personal),(bias_neutral),0.2340,0.7378,0.2146,0.917094,1.243012,0.041955,3.162619
1,(message_information),(bias_neutral),0.1294,0.7378,0.1130,0.873261,1.183602,0.017529,2.068822
5,(message_support),(bias_neutral),0.1842,0.7378,0.1502,0.815418,1.105202,0.014297,1.420507
6,(bias_partisan),(message_policy),0.2622,0.2822,0.1412,0.538520,1.908293,0.067207,1.555431
7,(message_policy),(bias_partisan),0.2822,0.2622,0.1412,0.500354,1.908293,0.067207,1.476647
2,(bias_neutral),(message_personal),0.7378,0.2340,0.2146,0.290865,1.243012,0.041955,1.080189
4,(bias_neutral),(message_support),0.7378,0.1842,0.1502,0.203578,1.105202,0.014297,1.024332
0,(bias_neutral),(message_information),0.7378,0.1294,0.1130,0.153158,1.183602,0.017529,1.028055


References:

‘Implementing Apriori algorithm in Python’ (2019) GeeksforGeeks, available: https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/ [accessed 29 Aug 2020].
    